In [1]:
video_path = './Data/videos/20240301_173556.mp4'
output_csv = './output'

In [2]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import deque

# Initialize YOLOv8 model
model = YOLO('yolov8s.pt')  # You can replace 'yolov8s.pt' with the path to your YOLOv8 model

def detect_persons(frame):
    results = model(frame)
    person_boxes = []
    
    for result in results[0].boxes:
        x1, y1, x2, y2 = map(int, result.xyxy[0])
        cls = int(result.cls)
        if cls == 0:  # Class ID 0 is for 'person'
            person_boxes.append((x1, y1, x2 - x1, y2 - y1))
    
    return person_boxes

def calculate_horizontal_velocity(current_positions, previous_positions):
    velocities = {}
    for i in current_positions:
        if i in previous_positions:
            velocities[i] = abs(current_positions[i][0] - previous_positions[i][0])
    return velocities

def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    tracker = cv2.TrackerKCF_create()
    prev_positions = {}
    max_velocity = 0
    bbox_highest_velocity = None

    tracks = {}
    track_id = 0
    frames_buffer = 30  # Number of frames to keep for each track

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Detect persons in the frame
        person_boxes = detect_persons(frame)
        
        # Track each person
        current_positions = {}
        for box in person_boxes:
            x, y, w, h = box
            center_x = x + w / 2
            center_y = y + h / 2

            # Find if this is a new track or existing one
            found = False
            for track_id, (prev_center_x, prev_center_y, track) in tracks.items():
                if abs(center_x - prev_center_x) < w and abs(center_y - prev_center_y) < h:
                    found = True
                    current_positions[track_id] = (center_x, box)
                    tracks[track_id] = (center_x, center_y, track)
                    break

            if not found:
                tracks[track_id] = (center_x, center_y, deque(maxlen=frames_buffer))
                current_positions[track_id] = (center_x, box)
                track_id += 1

        velocities = calculate_horizontal_velocity(current_positions, prev_positions)
        for track_id, velocity in velocities.items():
            if velocity > max_velocity:
                max_velocity = velocity
                bbox_highest_velocity = current_positions[track_id][1]

        prev_positions = current_positions

        # Draw bounding boxes and track IDs
        for track_id, (center_x, center_y, track) in tracks.items():
            if track_id in current_positions:
                box = current_positions[track_id][1]
                x, y, w, h = box
                cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
                cv2.putText(frame, f"ID: {track_id}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

        cv2.imshow('Frame', cv2.resize(frame, (1280, 720)))
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    return bbox_highest_velocity

# Example usage
#video_path = "path_to_your_video.mp4"
bbox = process_video(video_path)
print("Bounding box of the person with highest horizontal velocity:", bbox)


AttributeError: module 'cv2' has no attribute 'TrackerKCF_create'

: 